In [1]:
from urllib.request import urlopen
from PIL import Image
import timm

In [2]:
timm.__version__

'0.8.15dev0'

In [3]:
timm.list_models('*max*', pretrained=True)

['maxvit_base_tf_224.in1k',
 'maxvit_base_tf_384.in1k',
 'maxvit_base_tf_384.in21k_ft_in1k',
 'maxvit_base_tf_512.in1k',
 'maxvit_base_tf_512.in21k_ft_in1k',
 'maxvit_large_tf_224.in1k',
 'maxvit_large_tf_384.in1k',
 'maxvit_large_tf_384.in21k_ft_in1k',
 'maxvit_large_tf_512.in1k',
 'maxvit_large_tf_512.in21k_ft_in1k',
 'maxvit_nano_rw_256.sw_in1k',
 'maxvit_rmlp_base_rw_224.sw_in12k',
 'maxvit_rmlp_base_rw_224.sw_in12k_ft_in1k',
 'maxvit_rmlp_base_rw_384.sw_in12k_ft_in1k',
 'maxvit_rmlp_nano_rw_256.sw_in1k',
 'maxvit_rmlp_pico_rw_256.sw_in1k',
 'maxvit_rmlp_small_rw_224.sw_in1k',
 'maxvit_rmlp_tiny_rw_256.sw_in1k',
 'maxvit_small_tf_224.in1k',
 'maxvit_small_tf_384.in1k',
 'maxvit_small_tf_512.in1k',
 'maxvit_tiny_rw_224.sw_in1k',
 'maxvit_tiny_tf_224.in1k',
 'maxvit_tiny_tf_384.in1k',
 'maxvit_tiny_tf_512.in1k',
 'maxvit_xlarge_tf_384.in21k_ft_in1k',
 'maxvit_xlarge_tf_512.in21k_ft_in1k',
 'maxxvit_rmlp_nano_rw_256.sw_in1k',
 'maxxvit_rmlp_small_rw_256.sw_in1k',
 'maxxvitv2_nano_rw_2

In [4]:
timm.create_model("maxvit_xlarge_tf_512.in21k_ft_in1k", pretrained=True)

MaxxVit(
  (stem): Stem(
    (conv1): Conv2dSame(3, 192, kernel_size=(3, 3), stride=(2, 2))
    (norm1): BatchNormAct2d(
      192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): GELUTanh()
    )
    (conv2): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (stages): Sequential(
    (0): MaxxVitStage(
      (blocks): Sequential(
        (0): MaxxVitBlock(
          (conv): MbConvBlock(
            (shortcut): Downsample2d(
              (pool): AvgPool2dSame(kernel_size=(2, 2), stride=(2, 2), padding=(0, 0))
              (expand): Identity()
            )
            (pre_norm): BatchNormAct2d(
              192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
              (drop): Identity()
              (act): Identity()
            )
            (down): Identity()
            (conv1_1x1): Conv2d(192, 768, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (norm1): BatchNormAct2d(


In [5]:
img = Image.open(
    urlopen('https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/beignets-task-guide.png'))

model = timm.create_model('maxvit_xlarge_tf_512.in21k_ft_in1k', pretrained=True) # hf-hub:timm/
model = model.eval()

# get model specific transforms (normalization, resize)
data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=False)

output = model(transforms(img).unsqueeze(0))  # unsqueeze single image into batch of 1

top5_probabilities, top5_class_indices = torch.topk(output.softmax(dim=1) * 100, k=5)

In [6]:
from urllib.request import urlopen
from PIL import Image
import timm
import torch

In [7]:
img = Image.open(
    urlopen('https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/beignets-task-guide.png'))

model = timm.create_model('maxvit_xlarge_tf_512.in21k_ft_in1k', pretrained=True) # hf-hub:timm/
model = model.eval()

# get model specific transforms (normalization, resize)
data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=False)

output = model(transforms(img).unsqueeze(0))  # unsqueeze single image into batch of 1

top5_probabilities, top5_class_indices = torch.topk(output.softmax(dim=1) * 100, k=5)

In [8]:
top5_probabilities

tensor([[17.7779,  4.7684,  4.2991,  2.3935,  1.3077]],
       grad_fn=<TopkBackward0>)

In [9]:
top5_class_indices

tensor([[967, 968, 504, 415, 969]])

In [10]:
from fastai.vision.all import *


def get_pets(batch_size, img_size, seed):
    "The dog breeds pets datasets"
    dataset_path = untar_data(URLs.PETS)
    files = get_image_files(dataset_path/"images")
    dls = ImageDataLoaders.from_name_re(dataset_path, files, 
                                        pat=r'(^[a-zA-Z]+_*[a-zA-Z]+)', 
                                        valid_pct=0.2, 
                                        seed=seed, 
                                        bs=batch_size,
                                        item_tfms=Resize(img_size)) 
    return dls

In [11]:
config = SimpleNamespace(
    batch_size=64,
    img_size=224,
    seed=42,
    pretrained=False,
    model_name="regnetx_040",
    epochs=5)


def train(config):
    "Train the model using the supplied config"
    dls = get_pets(config.batch_size, config.img_size, config.seed)
    with wandb.init(project=PROJECT, group=GROUP, job_type=JOB_TYPE, config=config):
        cbs = [MixedPrecision(), WandbCallback(log_preds=False)]
        learn = vision_learner(dls, config.model_name, metrics=error_rate, 
                               cbs=cbs, pretrained=config.pretrained)
        learn.fine_tune(config.epochs)


train(config)

In [12]:
from urllib.request import urlopen
from PIL import Image
import timm
import torch
import wandb

In [13]:
config = SimpleNamespace(
    batch_size=64,
    img_size=224,
    seed=42,
    pretrained=False,
    model_name="regnetx_040",
    epochs=5)


def train(config):
    "Train the model using the supplied config"
    dls = get_pets(config.batch_size, config.img_size, config.seed)
    with wandb.init(project=PROJECT, group=GROUP, job_type=JOB_TYPE, config=config):
        cbs = [MixedPrecision(), WandbCallback(log_preds=False)]
        learn = vision_learner(dls, config.model_name, metrics=error_rate, 
                               cbs=cbs, pretrained=config.pretrained)
        learn.fine_tune(config.epochs)


train(config)

In [14]:
config = SimpleNamespace(
    batch_size=64,
    img_size=224,
    seed=42,
    pretrained=False,
    model_name="regnetx_040",
    epochs=5)


def train(config):
    "Train the model using the supplied config"
    dls = get_pets(config.batch_size, config.img_size, config.seed)
    with wandb.init(project="PROJECT", group='ambrosia_symbiosis', job_type='test_training', config=config):
        cbs = [MixedPrecision(), WandbCallback(log_preds=False)]
        learn = vision_learner(dls, config.model_name, metrics=error_rate, 
                               cbs=cbs, pretrained=config.pretrained)
        learn.fine_tune(config.epochs)


train(config)